# Rasa NLU

## Installation

Folgendes am besten in einem virtualenv / conda environment ausführen:

```
pip install rasa_core rasa_nlu[tensorflow] -U
pip install sklearn_crfsuite -U
```

In [1]:
import rasa_nlu
import rasa_core
import json

print("rasa_nlu: {} rasa_core: {}".format(rasa_nlu.__version__, rasa_core.__version__))

rasa_nlu: 0.15.0 rasa_core: 0.14.2


:0: UserWarning: You do not have a working installation of the service_identity module: 'No module named 'service_identity''.  Please install it from <https://pypi.python.org/pypi/service_identity> and make sure all of its dependencies are satisfied.  Without the service_identity module, Twisted can perform only rudimentary TLS client hostname verification.  Many valid certificate/hostname mappings may be rejected.


In [2]:
%matplotlib inline

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

def pprint(o):
    # small helper to make dict dumps a bit prettier
    print(json.dumps(o, indent=2))

In [3]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

In [4]:
# loading the nlu training samples
training_data = load_data('nlu.md')

INFO:rasa_nlu.training_data.loading:Training data format of nlu.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 22 (3 distinct intents)
	- Found intents: 'greet', 'restaurant_search', 'thankyou'
	- entity examples: 8 (2 distinct entities)
	- found entities: 'location', 'cuisine'



In [5]:
# trainer to educate our pipeline
trainer = Trainer(config.load("nlu_config.yml"))

In [6]:
# train the model!
interpreter = trainer.train(training_data, verbose=True)

# store it for future use
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

INFO:rasa_nlu.model:Starting to train component WhitespaceTokenizer
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component RegexFeaturizer
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component CRFEntityExtractor
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component EntitySynonymMapper
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component CountVectorsFeaturizer
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component EmbeddingIntentClassifier
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
INFO:rasa_nlu.classifiers.embedding_intent_classifier:Accuracy is updated every 10 epochs
Epochs: 100%|██████████| 300/300 [00:02<00:00, 109.15it/s, loss=0.084, acc=1.000]
INFO:rasa_nlu.classifiers.embedding_intent_classifier:Finished training embedding classifier, loss=0.084, train accuracy=1.000
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Successfully saved model into '/Users/das/mb17_synced/work/teaching/ss19/SDS/Sitzungen/04/Prep/models/nlu/default/current'


In [7]:
pprint(interpreter.parse("moderately priced restaurant in the west"))

{
  "intent": {
    "name": "restaurant_search",
    "confidence": 0.9701788425445557
  },
  "entities": [
    {
      "start": 36,
      "end": 40,
      "value": "west",
      "entity": "location",
      "confidence": 0.8219546816420032,
      "extractor": "CRFEntityExtractor"
    }
  ],
  "intent_ranking": [
    {
      "name": "restaurant_search",
      "confidence": 0.9701788425445557
    },
    {
      "name": "thankyou",
      "confidence": 0.05918114259839058
    },
    {
      "name": "greet",
      "confidence": 0.0
    }
  ],
  "text": "moderately priced restaurant in the west"
}


In [9]:
pprint(interpreter.parse("I am looking for a place to eat that is not in the south"))

{
  "intent": {
    "name": "restaurant_search",
    "confidence": 0.9789642095565796
  },
  "entities": [],
  "intent_ranking": [
    {
      "name": "restaurant_search",
      "confidence": 0.9789642095565796
    },
    {
      "name": "thankyou",
      "confidence": 0.0
    },
    {
      "name": "greet",
      "confidence": 0.0
    }
  ],
  "text": "I am looking for a place to eat that is not in the south"
}


In [11]:
pprint(interpreter.parse("good day"))

{
  "intent": {
    "name": "greet",
    "confidence": 0.9485673904418945
  },
  "entities": [],
  "intent_ranking": [
    {
      "name": "greet",
      "confidence": 0.9485673904418945
    },
    {
      "name": "thankyou",
      "confidence": 0.11189600825309753
    },
    {
      "name": "restaurant_search",
      "confidence": 0.0
    }
  ],
  "text": "good day"
}


In [12]:
from rasa_nlu.test import run_evaluation
import IPython
from IPython import display

run_evaluation('nlu.md', model_directory)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /Users/das/mb17_synced/work/teaching/ss19/SDS/Sitzungen/04/Prep/./models/nlu/default/current/component_5_EmbeddingIntentClassifier.ckpt
INFO:rasa_nlu.training_data.loading:Training data format of nlu.md is md
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 22 (3 distinct intents)
	- Found intents: 'greet', 'restaurant_search', 'thankyou'
	- entity examples: 8 (2 distinct entities)
	- found entities: 'location', 'cuisine'

INFO:rasa_nlu.test:Running model for predictions:
100%|██████████| 22/22 [00:00<00:00, 219.88it/s]
INFO:rasa_nlu.test:Intent evaluation results:
INFO:rasa_nlu.test:Intent Evaluation: Only considering those 22 examples that have a defined intent out of 22 examples
INFO:rasa_nlu.test:F1-Score:  1.0
INFO:rasa_nlu.test:Precision: 1.0
INFO:rasa_nlu.test:Accuracy:  1.0
INFO:rasa_nlu.test:Classification report: 


{'entity_evaluation': {'CRFEntityExtractor': {'accuracy': 1.0,
   'f1_score': 1.0,
   'precision': 1.0,
   'report': '              precision    recall  f1-score   support\n\n     cuisine       1.00      1.00      1.00         5\n    location       1.00      1.00      1.00         5\n   no_entity       1.00      1.00      1.00        78\n\n   micro avg       1.00      1.00      1.00        88\n   macro avg       1.00      1.00      1.00        88\nweighted avg       1.00      1.00      1.00        88\n'}},
 'intent_evaluation': {'accuracy': 1.0,
  'f1_score': 1.0,
  'precision': 1.0,
  'predictions': [{'confidence': 0.937021017074585,
    'intent': 'greet',
    'predicted': 'greet',
    'text': 'hey'},
   {'confidence': 0.9449281692504883,
    'intent': 'greet',
    'predicted': 'greet',
    'text': 'hello'},
   {'confidence': 0.9457494020462036,
    'intent': 'greet',
    'predicted': 'greet',
    'text': 'hi'},
   {'confidence': 0.9503194093704224,
    'intent': 'greet',
    'predict